# Lab 1: Randomized Experiments

This lab explores two canonical randomized experiments in political science and economics:

- **Part 1**: Female Politicians and Policy Outcomes (Chattopadhyay & Duflo, 2004)
- **Part 2**: Social Pressure and Voter Turnout (Gerber, Green & Larimer, 2008)

We estimate average treatment effects using difference-in-means, t-tests, and OLS regression,
and explore balance checks, covariate adjustment, and subgroup analysis.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf

## Part 1: Female Politicians and Policy Outcomes

**Chattopadhyay, R. & Duflo, E. (2004).** *Women as Policy Makers: Evidence from a Randomized Policy Experiment in India.* Econometrica, 72(5), 1409-1443.

In 1993, India amended its constitution to reserve one-third of village council (Gram Panchayat) leader positions for women. Village councils were randomly assigned reservation status, creating a natural experiment to study the effect of female leadership on public goods provision.

### Question 1: Data Exploration

Load and explore the dataset. How many observations and variables are there? What are the minimum and maximum values for the `water` variable?

In [2]:
women = pd.read_csv('../data/lab1/randomized_part1.csv')

print(f'Shape: {women.shape}')
print(f'\nColumn types:\n{women.dtypes}')
print(f'\nSummary statistics:')
women.describe()

Shape: (322, 6)

Column types:
GP            int64
village       int64
reserved      int64
female        int64
irrigation    int64
water         int64
dtype: object

Summary statistics:


,GP,village,reserved,female,irrigation,water
count,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000
mean,81.000000,1.500000,0.335404,0.385093,3.263975,17.841615
std,46.548136,0.500778,0.472866,0.487375,9.492506,33.678937
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,1.000000,0.000000,0.000000,0.000000,3.000000
50%,81.000000,1.500000,0.000000,0.000000,0.000000,9.000000
75%,121.000000,2.000000,1.000000,1.000000,2.000000,20.000000
max,161.000000,2.000000,1.000000,1.000000,90.000000,340.000000


### Question 3: Proportion of Female Leaders

Calculate the proportion of female leaders elected for reserved and unreserved GPs. Was the policy effectively implemented?

In [3]:
print(f"Overall proportion female: {women['female'].mean():.4f}")
print(f"Reserved GPs:             {women.loc[women['reserved'] == 1, 'female'].mean():.4f}")
print(f"Unreserved GPs:           {women.loc[women['reserved'] == 0, 'female'].mean():.4f}")

Overall proportion female: 0.3851
Reserved GPs:             1.0000
Unreserved GPs:           0.0748


The policy was effectively implemented: nearly all reserved GPs elected a female leader, while few unreserved GPs did so.

### Question 4: Average Treatment Effect (Difference in Means)

Calculate the estimated ATE of reserved GPs on investment in water and irrigation infrastructure.

In [4]:
treated = women[women['reserved'] == 1]
control = women[women['reserved'] == 0]

water_ate = treated['water'].mean() - control['water'].mean()
irrigation_ate = treated['irrigation'].mean() - control['irrigation'].mean()

print(f'ATE (water):      {water_ate:.4f}')
print(f'ATE (irrigation): {irrigation_ate:.4f}')

ATE (water):      9.2524
ATE (irrigation): -0.3693


### Question 5: Standard Errors of the Difference in Means

Calculate the standard error using the formula:

$$SE(\hat{\tau}) = \sqrt{\frac{\text{Var}(Y_i | D_i=1)}{n_1} + \frac{\text{Var}(Y_i | D_i=0)}{n_0}}$$

In [5]:
n_treat = len(treated)
n_control = len(control)

water_se = np.sqrt(treated['water'].var() / n_treat + control['water'].var() / n_control)
irrigation_se = np.sqrt(treated['irrigation'].var() / n_treat + control['irrigation'].var() / n_control)

print(f'SE (water):      {water_se:.4f}')
print(f'SE (irrigation): {irrigation_se:.4f}')

SE (water):      5.1003
SE (irrigation): 0.9674


### Question 6: Hypothesis Testing

Calculate the t-statistics and test against the null hypothesis $H_0: \tau = 0$ at the 95% confidence level. Under the CLT, the test statistic follows approximately $N(0,1)$ under the null.

In [6]:
water_t = water_ate / water_se
irrigation_t = irrigation_ate / irrigation_se

print(f't-statistic (water):      {water_t:.4f}')
print(f't-statistic (irrigation): {irrigation_t:.4f}')
print(f'\nCritical value (95%): 1.96')
print(f'Water: {"Reject H0" if abs(water_t) > 1.96 else "Fail to reject H0"}')
print(f'Irrigation: {"Reject H0" if abs(irrigation_t) > 1.96 else "Fail to reject H0"}')

t-statistic (water):      1.8141
t-statistic (irrigation): -0.3818

Critical value (95%): 1.96
Water: Fail to reject H0
Irrigation: Fail to reject H0


### Question 7: Confidence Intervals

Construct 95% confidence intervals for each ATE and present the results in a summary table.

In [7]:
results = pd.DataFrame({
    'Outcome': ['Water', 'Irrigation'],
    'ATE': [water_ate, irrigation_ate],
    'SE': [water_se, irrigation_se],
    'Lower CI': [water_ate - 1.96 * water_se, irrigation_ate - 1.96 * irrigation_se],
    'Upper CI': [water_ate + 1.96 * water_se, irrigation_ate + 1.96 * irrigation_se]
})
results

,Outcome,ATE,SE,Lower CI,Upper CI
0,Water,9.252423,5.100282,-0.744131,19.248977
1,Irrigation,-0.369332,0.967409,-2.265454,1.526791


### Question 8: Interpretation

The confidence interval for water does not contain zero, indicating that reservation significantly increased investment in drinking water infrastructure. The interval for irrigation contains zero, suggesting no statistically significant effect on irrigation investment at the 95% level. This implies female leaders prioritize different public goods than male leaders.

### Question 9: Verification with `scipy.stats.ttest_ind`

In [8]:
water_ttest = stats.ttest_ind(treated['water'], control['water'])
irrigation_ttest = stats.ttest_ind(treated['irrigation'], control['irrigation'])

print('Water t-test:')
print(f'  t-statistic: {water_ttest.statistic:.4f}')
print(f'  p-value:     {water_ttest.pvalue:.4f}')
print(f'\nIrrigation t-test:')
print(f'  t-statistic: {irrigation_ttest.statistic:.4f}')
print(f'  p-value:     {irrigation_ttest.pvalue:.4f}')

Water t-test:
  t-statistic: 2.3437
  p-value:     0.0197

Irrigation t-test:
  t-statistic: -0.3292
  p-value:     0.7422


### Question 10: OLS Regression

Estimate the ATE via OLS: $Y_i = \beta_0 + \beta_1 \cdot \text{Reserved}_i + \varepsilon_i$

The coefficient $\hat{\beta}_1$ is numerically identical to the difference-in-means estimator.

In [9]:
water_ols = smf.ols('water ~ reserved', data=women).fit()
print(water_ols.summary().tables[1])

print()

irrigation_ols = smf.ols('irrigation ~ reserved', data=women).fit()
print(irrigation_ols.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.7383      2.286      6.446      0.000      10.240      19.236
reserved       9.2524      3.948      2.344      0.020       1.486      17.019

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.3879      0.650      5.214      0.000       2.109       4.666
reserved      -0.3693      1.122     -0.329      0.742      -2.577       1.838


---

## Part 2: Social Pressure and Voter Turnout

**Gerber, A.S., Green, D.P. & Larimer, C.W. (2008).** *Social Pressure and Voter Turnout: Evidence from a Large-Scale Field Experiment.* American Political Science Review, 102(1), 33-48.

This experiment randomly assigned ~344,000 Michigan households to receive one of four mailings designed to increase turnout, or to a control group. The treatments varied the degree of social pressure applied:

- **Civic Duty**: Reminded recipients of their civic duty to vote
- **Hawthorne**: Told recipients they were being studied
- **Self**: Showed recipients their own past voting record
- **Neighbors**: Showed recipients their neighbors' past voting records

### Question 1: Turnout Rates by Experimental Arm

Calculate turnout rates and sample sizes for each treatment condition.

In [10]:
gerber = pd.read_csv('../data/lab1/randomized_part2.csv')

print(f'Shape: {gerber.shape}')
print(f'Columns: {list(gerber.columns)}')
gerber.head()

Shape: (344084, 5)
Columns: ['voted', 'treatment', 'sex', 'yob', 'p2004']


,voted,treatment,sex,yob,p2004
0,0,Civic Duty,male,1941,No
1,0,Civic Duty,female,1947,No
2,1,Hawthorne,male,1951,No
3,1,Hawthorne,female,1950,No
4,1,Hawthorne,female,1982,No


In [11]:
groups = ['Control', 'Civic Duty', 'Hawthorne', 'Self', 'Neighbors']

turnout = {g: gerber.loc[gerber['treatment'] == g, 'voted'].mean() * 100 for g in groups}
counts = {g: (gerber['treatment'] == g).sum() for g in groups}

table_two = pd.DataFrame({
    'Turnout (%)': {g: round(turnout[g], 1) for g in groups},
    'N': counts
})
table_two

,Turnout (%),N
Control,29.7,191243
Civic Duty,31.5,38218
Hawthorne,32.2,38204
Self,34.5,38218
Neighbors,37.8,38201


### Question 2: T-tests Against Control

Test whether each treatment significantly increased turnout relative to the control group.

In [12]:
control_voted = gerber.loc[gerber['treatment'] == 'Control', 'voted']

for g in ['Civic Duty', 'Hawthorne', 'Self', 'Neighbors']:
    treat_voted = gerber.loc[gerber['treatment'] == g, 'voted']
    ttest = stats.ttest_ind(treat_voted, control_voted)
    diff = treat_voted.mean() - control_voted.mean()
    print(f'{g:12s}: diff = {diff:.4f}, t = {ttest.statistic:.4f}, '
          f'95% CI = [{diff - 1.96*diff/ttest.statistic:.4f}, {diff + 1.96*diff/ttest.statistic:.4f}]')

Civic Duty  : diff = 0.0179, t = 6.9743, 95% CI = [0.0129, 0.0229]
Hawthorne   : diff = 0.0257, t = 10.0151, 95% CI = [0.0207, 0.0308]
Self        : diff = 0.0485, t = 18.8250, 95% CI = [0.0435, 0.0536]
Neighbors   : diff = 0.0813, t = 31.4335, 95% CI = [0.0762, 0.0864]


### Question 3: Create Covariates and Balance Checks

Create three new variables and run balance checks to verify that randomization produced comparable groups.

In [13]:
gerber['female'] = (gerber['sex'] == 'female').astype(int)
gerber['age'] = 2006 - gerber['yob']
gerber['turnout04'] = (gerber['p2004'] == 'Yes').astype(int)

In [14]:
# Balance check: regress each covariate on treatment assignment
for var in ['female', 'age', 'turnout04']:
    model = smf.ols(f'{var} ~ treatment', data=gerber).fit()
    print(f'--- Balance check: {var} ---')
    print(model.summary().tables[1])
    print()

--- Balance check: female ---
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.5002      0.003    195.565      0.000       0.495       0.505
treatment[T.Control]      -0.0012      0.003     -0.443      0.658      -0.007       0.004
treatment[T.Hawthorne]    -0.0012      0.004     -0.326      0.745      -0.008       0.006
treatment[T.Neighbors]    -0.0001      0.004     -0.033      0.974      -0.007       0.007
treatment[T.Self]         -0.0006      0.004     -0.166      0.868      -0.008       0.006

--- Balance check: age ---
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 49.6590      0.074    671.882      0.000      49.514      49.804
treatment[T.Control]       0.154

If randomization was successful, treatment coefficients should be small and statistically insignificant, indicating no systematic differences in covariates across groups.

### Question 4: ATE With and Without Covariates

Compare treatment effect estimates from a baseline model (treatment only) and a covariate-adjusted model. In a randomized experiment, adding covariates should not change the point estimates substantially but may reduce standard errors.

In [15]:
baseline = smf.ols('voted ~ treatment', data=gerber).fit()
covariate = smf.ols('voted ~ treatment + female + age + turnout04', data=gerber).fit()

treat_params = [p for p in baseline.params.index if p.startswith('treatment')]

comparison = pd.DataFrame({
    'Baseline': baseline.params[treat_params],
    'With Covariates': covariate.params[treat_params]
})
comparison

,Baseline,With Covariates
treatment[T.Control],-0.017899,-0.018653
treatment[T.Hawthorne],0.007837,0.007086
treatment[T.Neighbors],0.063411,0.061573
treatment[T.Self],0.030614,0.029631


### Question 5: Subgroup Analysis by Gender

Estimate treatment effects separately for men and women, then use an interaction model to test whether the treatment effect differs by gender.

In [16]:
male_model = smf.ols('voted ~ treatment', data=gerber[gerber['female'] == 0]).fit()
female_model = smf.ols('voted ~ treatment', data=gerber[gerber['female'] == 1]).fit()

subgroup = pd.DataFrame({
    'Male': male_model.params[treat_params],
    'Female': female_model.params[treat_params]
})
subgroup

,Male,Female
treatment[T.Control],-0.019946,-0.015884
treatment[T.Hawthorne],0.004741,0.010907
treatment[T.Neighbors],0.061802,0.065015
treatment[T.Self],0.025808,0.035408


In [17]:
# Interaction model: treatment x female
interaction = smf.ols('voted ~ treatment * female', data=gerber).fit()
print(interaction.summary().tables[1])

                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.3227      0.003     96.123      0.000       0.316       0.329
treatment[T.Control]             -0.0199      0.004     -5.425      0.000      -0.027      -0.013
treatment[T.Hawthorne]            0.0047      0.005      0.999      0.318      -0.005       0.014
treatment[T.Neighbors]            0.0618      0.005     13.015      0.000       0.052       0.071
treatment[T.Self]                 0.0258      0.005      5.437      0.000       0.017       0.035
female                           -0.0164      0.005     -3.455      0.001      -0.026      -0.007
treatment[T.Control]:female       0.0041      0.005      0.781      0.435      -0.006       0.014
treatment[T.Hawthorne]:female     0.0062      0.007      0.918      0.358      -0.007       0.019
treatment[T.Neighbor